In [2]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

ModuleNotFoundError: No module named 'findspark'

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("LinearRegWithSpark")\
        .getOrCreate()

In [3]:
dataset = spark.read.csv("data\Admission_Prediction.csv",header=True)

NameError: name 'spark' is not defined

In [4]:
dataset.show()

NameError: name 'dataset' is not defined

In [5]:
dataset.printSchema()

NameError: name 'dataset' is not defined

In [6]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

ModuleNotFoundError: No module named 'pyspark'

In [7]:
new_data.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [8]:
from pyspark.sql.functions import col, count, isnan, when

In [9]:
#let's check for missing values

In [10]:
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [11]:
from pyspark.ml.feature import Imputer

In [12]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_data)

imputed_data = model.transform(new_data)

In [13]:
#checking for null ir nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [14]:
features = imputed_data.drop('Chance of Admit')

In [15]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler(
    inputCols=features.columns,
    outputCol="features")

In [16]:
output = assembler.transform(imputed_data)

In [17]:
output= output.select("features", "Chance of Admit")

In [18]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [19]:
train_df.show()
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,93.0,1.0,1...|           0.46|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,99.0,2.0,2...|           0.57|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,99.0,2.0,2...|           0.61|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,99.0,1.0,1...|           0.53|
+--------------------+---------------+
only showing top 20 rows

+--------------------+---------------+

In [20]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit')
linear_model = lin_reg.fit(train_df)


In [21]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [0.001839160173256644,0.0034685604433953494,0.0019849135704206287,2.2081004658193452e-05,0.016441676636086394,0.11809415103249826,0.02650787281763461]
Intercept: -1.3214512480269756


In [22]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 0.057935
r2: 0.829099


In [23]:
# prediction

predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit","features").show()

+-------------------+---------------+--------------------+
|         prediction|Chance of Admit|            features|
+-------------------+---------------+--------------------+
|  0.486454409954852|           0.47|[290.0,100.0,1.0,...|
| 0.5027058674191538|           0.45|[290.0,104.0,4.0,...|
|0.42886741808072726|           0.46|[295.0,93.0,1.0,2...|
|0.48763502982797524|           0.47|[296.0,99.0,2.0,3...|
| 0.4721090308201181|           0.34|[297.0,96.0,2.0,2...|
| 0.5682165893804023|           0.34|[298.0,98.0,2.0,4...|
| 0.5613110894488953|           0.63|[299.0,100.0,3.0,...|
| 0.5068962304161391|           0.42|[299.0,100.0,3.0,...|
| 0.5627296508113693|           0.58|[300.0,99.0,1.0,1...|
| 0.6164188785498779|           0.62|[300.0,100.0,3.12...|
| 0.5583395482016189|           0.59|[300.0,101.0,3.0,...|
|  0.550377226207303|           0.56|[300.0,102.0,2.0,...|
| 0.5960554079395042|           0.63|[300.0,102.0,3.0,...|
| 0.6100323986044072|           0.71|[300.0,104.0,3.0,..

In [24]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Chance of Admit",metricName="r2")
print("R Squared (R2) on test data = %g" % pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.795967


In [25]:
spark.stop()